https://www.kaggle.com/c/state-farm-distracted-driver-detection  
based on https://github.com/jonas-pettersson/fast-ai/blob/master/statefarm.ipynb

In [1]:
sample_or_prod = True #True = sample, False = production
batch_size=8#reduced from 64 while running two models.

c0: safe driving  
c1: texting - right  
c2: talking on the phone - right  
c3: texting - left  
c4: talking on the phone - left  
c5: operating the radio  
c6: drinking  
c7: reaching behind  
c8: hair and makeup  
c9: talking to passenger  

In [2]:
cat_desc = ['safe driving', 'texting - right', 'talking on the phone - right', 
            'texting - left', 'talking on the phone - left', 'operating the radio', 
            'drinking', 'reaching behind', 'hair and makeup', 'talking to passenger']
print ("cat_desc:", type(cat_desc), len(cat_desc))

('cat_desc:', <type 'list'>, 10)


In [3]:
HOMEPATH = "/home/ubuntu/fastai/"
DATA_PATH = HOMEPATH + "data/state/"
print("HOMEPATH:", HOMEPATH)
print("DATA_PATH:", DATA_PATH)
import os
from __future__ import division, print_function


('HOMEPATH:', '/home/ubuntu/fastai/')
('DATA_PATH:', '/home/ubuntu/fastai/data/state/')


In [4]:
import os
from datetime import datetime
from glob import glob
from random import shuffle
from shutil import copyfile
import pandas as pd
import numpy as np
import random
import cv2

In [5]:
MODEL_PATH = DATA_PATH+"models/"
RESULTS_PATH = DATA_PATH+"results/"


SAMPLE_DATA_PATH = DATA_PATH + "sample/"#choose this for testing or above for production
SAMPLE_TEST_PATH = SAMPLE_DATA_PATH+"test/"
SAMPLE_TRAIN_PATH = SAMPLE_DATA_PATH + "train/"
SAMPLE_VALID_PATH = SAMPLE_DATA_PATH + "valid/"


TEST_PATH = DATA_PATH+"test/"
TRAIN_PATH = DATA_PATH + "train/"
VALID_PATH = DATA_PATH + "valid/"


if sample_or_prod:
    WORKING_DATA  = SAMPLE_DATA_PATH
    WORKING_TEST  = SAMPLE_TEST_PATH
    WORKING_TRAIN = SAMPLE_TRAIN_PATH
    WORKING_VALID = SAMPLE_VALID_PATH
    s_or_p = "_sample_"
else:
    WORKING_DATA  = DATA_PATH
    WORKING_TEST  = TEST_PATH
    WORKING_TRAIN = TRAIN_PATH
    WORKING_VALID = VALID_PATH
    s_or_p = "_prod_"

    
print ("WORKING_DATA:", WORKING_DATA)
print ("WORKING_TEST:", WORKING_TEST)
print ("WORKING_TRAIN:", WORKING_TRAIN)
print ("WORKING_VALID:", WORKING_VALID)
print ("s_or_p:", s_or_p)

WORKING_DATA: /home/ubuntu/fastai/data/state/sample/
WORKING_TEST: /home/ubuntu/fastai/data/state/sample/test/
WORKING_TRAIN: /home/ubuntu/fastai/data/state/sample/train/
WORKING_VALID: /home/ubuntu/fastai/data/state/sample/valid/
s_or_p: _sample_


In [6]:
os.chdir(DATA_PATH)
print (os.getcwd())
!tree -d

/home/ubuntu/fastai/data/state
.
├── models
├── results
├── sample
│   ├── test
│   │   └── unknown
│   ├── train
│   │   ├── c0
│   │   ├── c1
│   │   ├── c2
│   │   ├── c3
│   │   ├── c4
│   │   ├── c5
│   │   ├── c6
│   │   ├── c7
│   │   ├── c8
│   │   └── c9
│   └── valid
│       ├── c0
│       ├── c1
│       ├── c2
│       ├── c3
│       ├── c4
│       ├── c5
│       ├── c6
│       ├── c7
│       ├── c8
│       └── c9
├── test
│   └── unknown
├── train
│   ├── c0
│   ├── c1
│   ├── c2
│   ├── c3
│   ├── c4
│   ├── c5
│   ├── c6
│   ├── c7
│   ├── c8
│   └── c9
└── valid
    ├── c0
    ├── c1
    ├── c2
    ├── c3
    ├── c4
    ├── c5
    ├── c6
    ├── c7
    ├── c8
    └── c9

51 directories


In [7]:
def dirFileList(dir_path):
    return [name for name in os.listdir(dir_path) if os.path.isfile(os.path.join(dir_path, name))]


In [8]:
CATEGORIES = []


for i in range(10):
    CATEGORIES.append("c"+str(i)+"/")
print ("CATEGORIES:", CATEGORIES)

CATEGORIES: ['c0/', 'c1/', 'c2/', 'c3/', 'c4/', 'c5/', 'c6/', 'c7/', 'c8/', 'c9/']


In [9]:
UNKNOWN = 'unknown/'

DATA_DIR_LIST = [DATA_PATH, SAMPLE_DATA_PATH, SAMPLE_TEST_PATH, SAMPLE_TEST_PATH+UNKNOWN, SAMPLE_TRAIN_PATH, 
                 SAMPLE_VALID_PATH, TEST_PATH, TEST_PATH+UNKNOWN, TRAIN_PATH, VALID_PATH, MODEL_PATH, RESULTS_PATH]



for category in CATEGORIES:
    DATA_DIR_LIST.append(TRAIN_PATH+category)
    DATA_DIR_LIST.append(VALID_PATH+category)
    DATA_DIR_LIST.append(SAMPLE_TRAIN_PATH+category)
    DATA_DIR_LIST.append(SAMPLE_VALID_PATH+category)

In [10]:
def showLayersInfo(model):
    print ("Number of layers : ", len(model.layers))
    count = 0
    for layer in model.layers:
        print (count, type(layer), ", trainable:", layer.trainable)
        print ("input:", layer.input_shape, ", output:",layer.output_shape, ", len(weights)", len(layer.get_weights()), "\n")
        count +=1

In [11]:
def plot_history(history):
    # list all data in history
    print(history.history.keys())
    # summarize history for accuracy
    plt.plot(history.history['acc'])
    plt.plot(history.history['val_acc'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'validate'], loc='upper left')
    plt.show()
    # summarize history for loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'validate'], loc='upper left')
    plt.show()

In [12]:
def listDirsFileCount(DATA_DIR_LIST):
    DATA_DIR_LIST = sorted(DATA_DIR_LIST)
    for dir_ in DATA_DIR_LIST:
        print (dir_, len(dirFileList(dir_)))
        
#NB: could have used global DATA_DIR_LIST  to access global variable DATA_DIR_LIST 
# without having to pass DATA_DIR_LIST as an arg, but this gives more flexability.

In [13]:
listDirsFileCount(DATA_DIR_LIST)

/home/ubuntu/fastai/data/state/ 5
/home/ubuntu/fastai/data/state/models/ 9
/home/ubuntu/fastai/data/state/results/ 2
/home/ubuntu/fastai/data/state/sample/ 0
/home/ubuntu/fastai/data/state/sample/test/ 0
/home/ubuntu/fastai/data/state/sample/test/unknown/ 15945
/home/ubuntu/fastai/data/state/sample/train/ 0
/home/ubuntu/fastai/data/state/sample/train/c0/ 347
/home/ubuntu/fastai/data/state/sample/train/c1/ 314
/home/ubuntu/fastai/data/state/sample/train/c2/ 323
/home/ubuntu/fastai/data/state/sample/train/c3/ 325
/home/ubuntu/fastai/data/state/sample/train/c4/ 323
/home/ubuntu/fastai/data/state/sample/train/c5/ 322
/home/ubuntu/fastai/data/state/sample/train/c6/ 324
/home/ubuntu/fastai/data/state/sample/train/c7/ 278
/home/ubuntu/fastai/data/state/sample/train/c8/ 268
/home/ubuntu/fastai/data/state/sample/train/c9/ 298
/home/ubuntu/fastai/data/state/sample/valid/ 0
/home/ubuntu/fastai/data/state/sample/valid/c0/ 153
/home/ubuntu/fastai/data/state/sample/valid/c1/ 138
/home/ubuntu/fastai/

In [14]:
os.chdir(HOMEPATH)
print ("current working directory:", os.getcwd())

%matplotlib inline
from __future__ import print_function, division
#path = "data/state/"
path = "data/state/sample/"
import utils; reload(utils)
from utils import *
from IPython.display import FileLink

current working directory: /home/ubuntu/fastai


WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5110)
Using Theano backend.


In [15]:
batches = get_batches(WORKING_TRAIN, batch_size=batch_size)
val_batches = get_batches(WORKING_VALID, batch_size=batch_size*2, shuffle=False)
test_batches = get_batches(WORKING_TEST, batch_size=1, shuffle=False)
print (WORKING_TRAIN, type(batches))
print (WORKING_VALID, type(val_batches))
print (WORKING_TEST, type(test_batches))


#https://keras.io/preprocessing/image/
#https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/DirectoryIterator
#NB: utils.get_batches has default target_size=(224,224)

Found 3122 images belonging to 10 classes.
Found 1369 images belonging to 10 classes.
Found 15945 images belonging to 1 classes.
/home/ubuntu/fastai/data/state/sample/train/ <class 'keras.preprocessing.image.DirectoryIterator'>
/home/ubuntu/fastai/data/state/sample/valid/ <class 'keras.preprocessing.image.DirectoryIterator'>
/home/ubuntu/fastai/data/state/sample/test/ <class 'keras.preprocessing.image.DirectoryIterator'>


In [16]:
(val_classes, trn_classes, val_labels, trn_labels, val_filenames, filenames, test_filename) = get_classes(WORKING_DATA)
print ("val_classes", type(val_classes), val_classes.shape)
print ("trn_classes", type(trn_classes), trn_classes.shape)
print ("val_labels", type(val_labels), val_labels.shape)
print ("trn_labels", type(trn_labels), trn_labels.shape)
print ("val_filenames", type(val_filenames), len(val_filenames))
print ("filenames", type(filenames), len(filenames))
print ("test_filename", type(test_filename), len(test_filename))

Found 3122 images belonging to 10 classes.
Found 1369 images belonging to 10 classes.
Found 15945 images belonging to 1 classes.
val_classes <type 'numpy.ndarray'> (1369,)
trn_classes <type 'numpy.ndarray'> (3122,)
val_labels <type 'numpy.ndarray'> (1369, 10)
trn_labels <type 'numpy.ndarray'> (3122, 10)
val_filenames <type 'list'> 1369
filenames <type 'list'> 3122
test_filename <type 'list'> 15945


In [17]:
print ("test_filename[0]", test_filename[0])

test_filename[0] unknown/img_92913.jpg


In [18]:
print ("val_classes:", type(val_classes[0]), val_classes[0], np.unique(val_classes))
print ("trn_classes:", type(trn_classes[0]), trn_classes[0], np.unique(trn_classes))
print ("val_labels:", type(val_labels[0]), val_labels[0], np.unique(val_labels))
print ("trn_labels:", type(trn_labels[0]), trn_labels[0], np.unique(trn_labels))

val_classes: <type 'numpy.int32'> 0 [0 1 2 3 4 5 6 7 8 9]
trn_classes: <type 'numpy.int32'> 0 [0 1 2 3 4 5 6 7 8 9]
val_labels: <type 'numpy.ndarray'> [ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.] [ 0.  1.]
trn_labels: <type 'numpy.ndarray'> [ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.] [ 0.  1.]


In [19]:
temp_path = TRAIN_PATH+CATEGORIES[0]
print ("temp_path:", temp_path)
files = dirFileList(temp_path)

image_sizes = set([])
for i in range(len(files)):
    img_shape = cv2.imread(temp_path+files[i]).shape
    #print ("i:", i, ":", img_shape)
    image_sizes.add(img_shape)
print ("image_sizes:", image_sizes)
print ("No. of disctinct image sizes:", len(image_sizes))

temp_path: /home/ubuntu/fastai/data/state/train/c0/
image_sizes: set([(480, 640, 3)])
No. of disctinct image sizes: 1


In [20]:
trn = get_data(WORKING_TRAIN)
val = get_data(WORKING_VALID)
test = get_data(WORKING_TEST)
print ("trn:", type(trn))
print ("val:", type(val))
print ("test:", type(test))

Found 3122 images belonging to 10 classes.
Found 1369 images belonging to 10 classes.
Found 15945 images belonging to 1 classes.
trn: <type 'numpy.ndarray'>
val: <type 'numpy.ndarray'>
test: <type 'numpy.ndarray'>


In [21]:
print ("trn:", type(trn))
print ("val:", type(val))
print ("test:", type(test))

trn: <type 'numpy.ndarray'>
val: <type 'numpy.ndarray'>
test: <type 'numpy.ndarray'>


In [22]:
vgg = Vgg16()
model=vgg.model
last_conv_idx = [i for i,l in enumerate(model.layers) if type(l) is Convolution2D][-1]
conv_layers = model.layers[:last_conv_idx+1]

In [23]:
print ("last_conv_idx:", last_conv_idx)

last_conv_idx: 30


In [24]:
conv_model = Sequential(conv_layers)

Next three blocks are pre-computing the output of conv_layers.

In [25]:
startTime= datetime.now()
print ("startTime:", startTime)

conv_feat = conv_model.predict_generator(batches, batches.nb_sample)
#batches = get_batches(WORKING_TRAIN, batch_size=batch_size) : Found 3122 images belonging to 10 classes.
print ("conv_feat:", type(conv_feat))

timeElapsed=datetime.now()-startTime
print('Time elapsed (hh:mm:ss.ms) {}'.format(timeElapsed))


startTime: 2018-01-05 01:24:05.478651
Time elapsed (hh:mm:ss.ms) 0:02:39.023115


In [26]:
startTime= datetime.now()
print ("startTime:", startTime)

conv_val_feat = conv_model.predict_generator(val_batches, val_batches.nb_sample)
#val_batches = get_batches(WORKING_VALID, batch_size=batch_size*2, shuffle=False):Found 1369 images belonging to 10 classes.
print ("conv_val_feat:", type(conv_val_feat))

timeElapsed=datetime.now()-startTime
print('Time elapsed (hh:mm:ss.ms) {}'.format(timeElapsed))


startTime: 2018-01-05 01:26:44.514200
Time elapsed (hh:mm:ss.ms) 0:01:03.444365


In [27]:
startTime= datetime.now()
print ("startTime:", startTime)

conv_test_feat = conv_model.predict_generator(test_batches, test_batches.nb_sample)
#test_batches = get_batches(WORKING_TEST, batch_size=1, shuffle=False) : Found 15945 images belonging to 1 classes.
print ("conv_test_feat:", type(conv_test_feat))

timeElapsed=datetime.now()-startTime
print('Time elapsed (hh:mm:ss.ms) {}'.format(timeElapsed))


startTime: 2018-01-05 01:27:47.970731
Time elapsed (hh:mm:ss.ms) 0:24:44.554337


In [28]:
print ("output shape from conv_layers: ", conv_layers[-1].output_shape[1:])

output shape from conv_layers:  (512, 14, 14)


In [29]:
def get_bn_layers(p):
    #create list of layers to accept input from conv_layers model.
    #output layer is Dense with 10 categories.
    return [
        MaxPooling2D(input_shape=conv_layers[-1].output_shape[1:]),
        Flatten(),
        Dropout(p/2),
        Dense(128, activation='relu'),
        BatchNormalization(),
        Dropout(p/2),
        Dense(128, activation='relu'),
        BatchNormalization(),
        Dropout(p),
        Dense(10, activation='softmax')
        ]

In [30]:
p=0.8

In [31]:
bn_model = Sequential(get_bn_layers(p))
bn_model.compile(Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [34]:
bn_model.fit(conv_feat, trn_labels, batch_size=batch_size, nb_epoch=1, 
             validation_data=(conv_val_feat, val_labels))

Train on 3122 samples, validate on 1369 samples
Epoch 1/1
3122/3122 [==============================] - 6s - loss: 4.2499 - acc: 0.1006 - val_loss: 2.5655 - val_acc: 0.1030


In [35]:
bn_model.optimizer.lr=0.01

In [36]:
history = bn_model.fit(conv_feat, trn_labels, batch_size=batch_size, nb_epoch=2, 
             validation_data=(conv_val_feat, val_labels))
#very low accuracy returned on sample: loss: 3.2319 - acc: 0.1015 - val_loss: 2.3709 - val_acc: 0.0920

Train on 3122 samples, validate on 1369 samples
Epoch 1/2
3122/3122 [==============================] - 6s - loss: 3.2319 - acc: 0.1015 - val_loss: 2.3709 - val_acc: 0.0920
Epoch 2/2
3122/3122 [==============================] - 7s - loss: 2.6921 - acc: 0.1166 - val_loss: 2.3243 - val_acc: 0.1059


In [ ]:
plot_history(history)

In [ ]:
#bn_model.save_weights(MODEL_PATH+'conv8.h5')

Now precompute data augmentation and perform dropout.

In [37]:
gen_t = image.ImageDataGenerator(rotation_range=15, height_shift_range=0.05, 
                shear_range=0.1, channel_shift_range=20, width_shift_range=0.1)
da_batches = get_batches(path+'train', gen_t, batch_size=batch_size, shuffle=False)

Found 3122 images belonging to 10 classes.


In [38]:
da_conv_feat = conv_model.predict_generator(da_batches, da_batches.nb_sample*5)

MemoryError: 

In [ ]:
#save_array(MODEL_PATH+'da_conv_feat2.dat', da_conv_feat)

In [ ]:
#da_conv_feat = load_array(MODEL_PATH+'results/da_conv_feat2.dat')

In [ ]:
#add the real training data as well in its non-augmented form.
da_conv_feat = np.concatenate([da_conv_feat, conv_feat])
print ("da_conv_feat:", type(da_conv_feat), da_conv_feat.shape)

In [ ]:
#copy the labels 6 times too. 
#NB: assumes the augmentation process replicates the original data abc abc abc not aaa bbb ccc
da_trn_labels = np.concatenate([trn_labels]*6)
print ("da_trn_labels:", type(da_trn_labels), da_trn_labels.shape)

In [ ]:
def get_bn_da_layers(p):
    return [
        MaxPooling2D(input_shape=conv_layers[-1].output_shape[1:]),
        Flatten(),
        Dropout(p),
        Dense(256, activation='relu'),
        BatchNormalization(),
        Dropout(p),
        Dense(256, activation='relu'),
        BatchNormalization(),
        Dropout(p),
        Dense(10, activation='softmax')
        ]